# Lab Exercise 1. Scraping Static Websites


This is the warmup task for the first laboratory exercise. It consists of scraping static Websites with BeautifulSoap.

 It should be completed at home and presented at the laboratory.

**Total points: 2**

### Task Description

Scrape the information about the products on the following page:
https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/

For each product scrape:


*   Product title (selector `'.wd-entities-title'`)
*   Product regular price (selector `'.woocommerce-Price-amount'`)
*   Product discount price (if available), same selector as regular price
*   URL to the product page
*   Add to cart button URL

***Help: There are multiple product pages, for each page you need to send a separate request***


Save the results as a DataFrame object

You can add as many code cells as you need.

________________________________________________________________

### Requirements

Import libraries and modules that you are going to use

In [1]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup

### Send HTTP request to the target Website

In [2]:
url = "https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/1/?per_page=36"
response =  rq.get(url)

check the response status code

In [3]:
print(response)

<Response [200]>


### Parse the response content with BeautifulSoap

In [4]:
soup = BeautifulSoup(response.text, 'html.parser')

### Extract data from the BeautifulSoap object using any selectors, attribute identifiers, etc.

* Product title (selector '.wd-entities-title')
* Product regular price (selector '.woocommerce-Price-amount')
* Product discount price (if available), same selector as regular price
* URL to the product page
* Add to cart button URL

In [5]:
products = soup.find_all('div', {'class':'product-grid-item'})
print(len(products))

36


Repeat the extraction process for each page of products

In [6]:
products = []
num_of_pages = int(soup.find('ul', {'class':'page-numbers'}).find_all('li')[-2].text)
for i in range(1, num_of_pages + 1):
  url = f'https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/page/{i}/?per_page=36'
  response = rq.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  products += soup.find_all('div', {'class':'product-grid-item'})

print(len(products))

320


### Create a pandas DataFrame with the scraped products

In [7]:
data = pd.DataFrame(columns=['Title', 'Regular Price', 'Discount Price', 'Product URL', 'Cart URL'])

for product in products:
  title = product.find('h3', {'class':'wd-entities-title'}).text
  prices = product.find_all('span', {'class':'woocommerce-Price-amount'})
  regular_price = int(prices[0].text.replace("ден", "").replace(".", ""))
  discounted_price = int(prices[1].text.replace("ден", "").replace(".", "")) if len(prices) ==  2 else None
  product_url = product.find('h3', {'class':'wd-entities-title'}).find('a').get('href')
  cart_url = 'https://clevershop.mk/product-category/mobilni-laptopi-i-tableti/' + product.find('div', {'class':'wd-add-btn'}).find('a').get('href')
  prod = {'Title': title, 'Regular Price': regular_price, 'Discount Price': discounted_price, 'Product URL': product_url, 'Cart URL': cart_url}
  data = data._append(prod, ignore_index=True)

data

,Title,Regular Price,Discount Price,Product URL,Cart URL
0,Acer A315-23-A7KD,17590,None,https://clevershop.mk/product/acer-a315-23-a7kd/,https://clevershop.mk/product-category/mobilni...
1,Acer A315-23-R5P2,27490,None,https://clevershop.mk/product/acer-a315-23-r5p2/,https://clevershop.mk/product-category/mobilni...
2,ACER Aspire 1 A115-22,18999,15999,https://clevershop.mk/product/acer-aspire-1-nx...,https://clevershop.mk/product-category/mobilni...
3,Acer Aspire 3 A315-23-R26A,29990,None,https://clevershop.mk/product/acer-aspire-3-a3...,https://clevershop.mk/product-category/mobilni...
4,Acer Aspire 3 A315-58-33WK,24490,None,https://clevershop.mk/product/21498/,https://clevershop.mk/product-category/mobilni...
...,...,...,...,...,...
315,Monitor 27 Philips 272E1GAJ/00 VA 1ms 144Hz,12890,None,https://clevershop.mk/product/monitor-27-phili...,https://clevershop.mk/product-category/mobilni...
316,Philips 24″ 243V7QDSB,8390,None,https://clevershop.mk/product/philips-24%e2%80...,https://clevershop.mk/product-category/mobilni...
317,Philips 27″ 278E1A/00 4K UHD IPS,18990,None,https://clevershop.mk/product/hp-27%e2%80%b3-2...,https://clevershop.mk/product-category/mobilni...
318,Philips 279C9-00 MON LED 27″ 3840 x 2160 5Ms 6...,26990,None,https://clevershop.mk/product/philips-279c9-00...,https://clevershop.mk/product-category/mobilni...


Save the dataframe as `.csv`

In [8]:
data.to_csv('clevershop_products.csv', index=False)